```
Client ID
RFQKGJMFRIY0H5WWUHHEN5YHNJMSEPQEQTCQOPMNIQT1R0FF
Client Secret
3GAEJPFACNZJM2NIG0JZINVV1PIE2O0OTVBVLRX0KC4OVAUQ
```

In [1]:
# import needed modules

import pandas as pd
import json, requests
import numpy as np
import requests
from geopy.geocoders import Nominatim

from IPython.display import Image
from IPython.core.display import HTML

from pandas.io.json import json_normalize

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

import folium

In [2]:
with open('hoods.geojson') as json_data:
    nyc = json.load(json_data)

In [3]:
hood_data = nyc['features']

In [4]:
hood_data[0]

{'type': 'Feature',
 'properties': {'stacked': '1',
  'name': 'Wakefield',
  'annoline1': 'Wakefield',
  'annoline3': '',
  'objectid': '1',
  'annoangle': '0.0',
  'annoline2': '',
  'borough': 'Bronx'},
 'geometry': {'type': 'Point',
  'coordinates': [-73.8472005205491, 40.89470517661004]}}

In [5]:
columns = ['borough', 'neighborhood', 'latitude', 'longitude']

hoods = pd.DataFrame(columns=columns)

In [6]:
for data in hood_data:
    borough = data['properties']['borough']
    hood_name = data['properties']['name']
    
    hood_latlon = data['geometry']['coordinates']
    hood_lat = hood_latlon[1]
    hood_lon = hood_latlon[0]
    
    hoods = hoods.append({'borough': borough,
                          'neighborhood': hood_name,
                          'latitude': hood_lat,
                          'longitude': hood_lon}, ignore_index=True)

In [7]:
address = 'New York City, NY'

geoloc = Nominatim(user_agent='ny')
location = geoloc.geocode(address)
lat = location.latitude
lon = location.longitude

In [8]:
map_nyc = folium.Map(location=[lat, lon], zoom_start=15)

for lat, lng, borough, neighborhood in zip(hoods['latitude'], hoods['longitude'], hoods['borough'], hoods['neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyc)  
    
map_nyc

In [10]:
hoods.to_csv("hoods.csv")

In [19]:
from config import client_id, client_secret

CLIENT_ID = client_id
CLIENT_SECRET = client_secret
VERSION = '20180605'

In [20]:
venue_id = '4fa862b3e4b0ebff2f749f06' 

url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()

print(result['response']['venue']['rating'])


7.0


In [ ]:
def getBurgers(labels, lats, lons):
    
    burgers = []
    
    for label, lat, lon in zip(labels, lats, lons):
        
        print(label)
        
        url =  'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&query=burgers&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lon, 
            500, 
            100)
        
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        burgers.append([(
            label,
            lat,
            lon,
            v['venue']['id'],
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    burgers_df = pd.DataFrame([i for burger in burgers])
    burgers_df.columns = ['Neighborhood', 
              'Neighborhood Latitude', 
              'Neighborhood Longitude',
              'Venue ID',
              'Venue', 
              'Venue Latitude', 
              'Venue Longitude', 
              'Venue Category']

    return(burgers_df)    
    

In [17]:
# code to extract ratings with ID
# need to convert to loop and reinsert to dataframe w/ correspondence

venue_id = '4fa862b3e4b0ebff2f749f06' 

url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()

print(result['response']['venue']['rating'])

7.0


In [ ]:
ny_burgers = getBurgers(labels=hoods['neighborhood'],
                       lats=hoods['latitude'],
                       lons=hoods['longitude'])

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park


In [25]:
ny_burgers.shape

NameError: name 'ny_burgers' is not defined

In [62]:
# categories for burgers

len(ny_burgers['Venue Category'].unique())

68

In [93]:
ny_burgers.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Co-op City,40.874294,-73.829939,4be2b79d660ec9284d04ca3b,Townhouse Restaurant,40.876086,-73.828868,Restaurant
1,Eastchester,40.887556,-73.827806,4c406f2e6c1a9521a042b96a,McDonald's,40.885636,-73.826840,Burger Joint
2,Kingsbridge,40.881687,-73.902818,55568a71498e3524400a356e,Smashburger,40.884221,-73.900333,Burger Joint
3,Kingsbridge,40.881687,-73.902818,4b3789f3f964a520f34125e3,Riverdale Diner,40.885183,-73.899484,Diner
4,Kingsbridge,40.881687,-73.902818,4b9c9c6af964a520b27236e3,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant


In [76]:
most_burgers = ny_burgers.groupby('Neighborhood').count().reset_index().sort_values('Venue', ascending=False)

most_burgers.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
97,Midtown South,61,61,61,61,61,61
96,Midtown,59,59,59,59,59,59
53,Financial District,51,51,51,51,51,51
29,Civic Center,30,30,30,30,30,30
161,Upper West Side,30,30,30,30,30,30


In [81]:
# work in progress

burgers_onehot = pd.get_dummies(ny_burgers[['Venue Category']], prefix="", prefix_sep="")

burgers_onehot['Neighborhood'] = ny_burgers[['Neighborhood']]

burgers_onehot.set_index('Neighborhood')


,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,BBQ Joint,Bakery,Bar,Beer Garden,Belgian Restaurant,Brazilian Restaurant,...,Snack Place,Southern / Soul Food Restaurant,Sports Bar,Steakhouse,Swiss Restaurant,Tapas Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Whisky Bar,Wings Joint
Neighborhood,,,,,,,,,,,,,,,,,,,,,
Co-op City,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Eastchester,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Kingsbridge,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Kingsbridge,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Kingsbridge,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Kingsbridge,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Kingsbridge,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Kingsbridge,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Marble Hill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
# work in progress

kclusters = 10

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(burgers_onehot)

kmeans.labels_[:]


ValueError: could not convert string to float: 'Middle Village'